In [ ]:
#!/usr/bin/python3
# coding: utf-8
# Hokkaido

In [ ]:
import codecs
from datetime import datetime as dt
from datetime import timedelta as td
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import matplotlib
import sys
if "ipy" not in sys.argv[0]:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from PIL import Image
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly, make_exp_fit_graph
from hokkaidomap import sub_prefs, get_sub_code, get_hokkaido, \
    make_hokkaido_choropleth, make_hokkaido_plotly
from urllib.request import urlretrieve

In [ ]:
patients_file = "010006_hokkaido_covid19_patients.csv"

In [ ]:
u='https://www.harp.lg.jp/opendata/api/package_show?id=752c577e-0cbe-46e0-bebd-eb47b71b38bf'
urlretrieve(u, 'hokkaido-api.json')
f=codecs.open("hokkaido-api.json", encoding='utf-8')
x=json.load(f)
f.close()
#updated = x['result']['updated'][:16]
with open("hokkaido.prev.tmp", "rt") as f:
    prev = f.read().rstrip()
#today = dt.now().isoformat()[:16]

for i in x['result']['resources']:
    if i['filename'] == patients_file:
        updated = i['updated'][:16]
        break

print(updated, prev)
if updated == prev:
    print("maybe the same data, nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

with open("hokkaido.prev.tmp", "wt") as f:
    f.write(updated)

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
# 北海道 OpenData を参照する
# https://www.harp.lg.jp/opendata/dataset/1369.html
# 北海道は面積が広いため、振興局別に集計を行う
base_uri = "https://www.harp.lg.jp/opendata/dataset/1369/resource/3132/"
download_if_needed(base_uri, patients_file)
base_uri = "https://www.harp.lg.jp/opendata/dataset/1369/resource/2853/"
data_file = "covid19_data.csv"
download_if_needed(base_uri, data_file)

In [ ]:
patients = []
# CSVデータを整形する
with codecs.open(patients_file, encoding="shift-jis") as f:
    l = f.readline() # 先頭行をスキップ
    while l:
        l = f.readline().replace("\r\n", "")
        arr = l.split(',')
        if len(arr) == 16:
            patients.append(
                [
                    code2int(arr[0]), # No
                    get_sub_code(arr[6]), # sub-pref code
                    arr[6], # sub-pref
                    dt.strptime(arr[4], "%Y-%m-%d"), # press date
                    arr[5], # infected 
                    arr[7], # Age
                    arr[8], # Sex
                ]
            )

print("Total: {}".format(len(patients)))
print(patients[-1][3])

In [ ]:
# Pandas DataFrame を作成する
df = pd.DataFrame(patients, columns=['No', 'Code', 'State', 'Date', 'DoW', 'Age', 'Sex'])

In [ ]:
# Daily の新規感染者を集計する
daily_new = df.groupby('Date').size()
print("Daily New: {}".format(daily_new[-1]))
# duration
frm_date = daily_new.index[0]
end_date = daily_new.index[-1]
print("From: {} To: {}".format(frm_date, end_date))
# moving average by week
ave_mov_days = 7
# 移動平均を算出する
mov_mean = daily_new.rolling(ave_mov_days).mean()

In [ ]:
# 4 weeks later
xbins = daily_new.index.tolist()
days2pred = 2 * ave_mov_days # 2 weeks
# 2週間先の日付列を取得する
two_weeks_later = pd.date_range(xbins[-1] + td(days=1), xbins[-1] + td(days=days2pred)).to_pydatetime()
xbins_pred = xbins.copy()
xbins_pred.extend(two_weeks_later)

X = np.arange(0, len(daily_new.index.values))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]
y_gpr = get_gpr_predict(X, daily_new.values, X_pred, 10, 10, 10)

In [ ]:
save_jma_data("sapporo-jma.csv", city_code="s47412")
weather_sapporo = parse_jma_csv("sapporo-jma.csv")
print("Loaded {} data.".format(len(weather_sapporo)))
df_weather = pd.DataFrame(weather_sapporo, columns=['Date', 'Temp', 'RH', 'VP', 'AP', 'AH', 'Fd'])
fig = px.scatter(df_weather, x='Date', y=['AH'])
fig.update_layout(template='plotly_dark')
if "ipy" in sys.argv[0]:
    fig.show()

In [ ]:
np_weather = np.array(weather_sapporo)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=xbins, y=daily_new, mode='markers', name='新規',
    marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(
    x=xbins, y=mov_mean, name='7日移動平均', opacity=0.6),
    secondary_y=False)
fig.add_trace(go.Scatter(
    x=xbins_pred, y=y_gpr, mode='lines', name='予測',
    line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Scatter(
    x=np_weather[:, 0], # 日付
    y=np_weather[:, 1], # 気温
    name="札幌平均気温",
    line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=(xbins[4], xbins_pred[-1])),
    yaxis=dict(title='人数', type="log"),
    yaxis2=dict(title='札幌平均気温'),
    title='北海道 新型コロナ 新規感染者数/札幌平均気温({})'.format(today_str),
)
show_and_save_plotly(fig, "hokkaido.jpg", js=False, show=False)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "北海道 新型コロナ 新規感染者数/気温(" + today_str + " 時点)"
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
tw_body += "(気象業務法第13～24条に接触するため、予報を含まない前日までの気温を表示) "
tw = get_twitter()
tweet_with_image(tw, "docs/images/hokkaido.jpg", tw_body)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=xbins, y=daily_new, mode='markers', name='新規',
    marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(
    x=xbins, y=mov_mean, name='7日移動平均', opacity=0.6),
    secondary_y=False)
fig.add_trace(go.Scatter(
    x=xbins_pred, y=y_gpr, mode='lines', name='予測',
    line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Scatter(
    x=np_weather[:, 0], # 日付
    y=np_weather[:, 5], # 絶対湿度
    name="絶対湿度",
    line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=(xbins[4], xbins_pred[-1])),
    yaxis=dict(title='人数', type="log"),
    yaxis2=dict(title='札幌平均容積絶対湿度 [g/㎥]'),
    title='北海道 新型コロナ 新規感染者数/札幌絶対湿度({})'.format(today_str),
)
show_and_save_plotly(fig, "hokkaido-ah.jpg", js=False, show=False)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "北海道 新型コロナ 新規感染者数/絶対湿度(" + today_str + " 時点)"
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
tw_body += "(気象業務法第13～24条に接触するため、予報を含まない前日までの絶対湿度を表示) "
#tw = get_twitter()
tweet_with_image(tw, "docs/images/hokkaido-ah.jpg", tw_body)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=xbins, y=daily_new, mode='markers', name='新規',
    marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(
    x=xbins, y=mov_mean, name='7日移動平均', opacity=0.6),
    secondary_y=False)
fig.add_trace(go.Scatter(
    x=xbins_pred, y=y_gpr, mode='lines', name='予測',
    line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Scatter(
    x=np_weather[:, 0], # 日付
    y=np_weather[:, 6], # 絶対湿度
    name="空気抵抗力",
    line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=(xbins[4], xbins_pred[-1])),
    yaxis=dict(title='人数', type="log"),
    yaxis2=dict(title='ウィルスに働く空気抵抗力'),
    title='北海道 新型コロナ 新規感染者数/空気抵抗力({})'.format(today_str),
)
show_and_save_plotly(fig, "hokkaido-fd.jpg", js=False, show=False)

In [ ]:
if False:
    tw_body = "北海道 新型コロナ 新規感染者数/空気抵抗力(" + today_str + " 時点)"
    tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
    tw_body += "(ウィルス微粒子に働く空気抵抗力をプロット、低いほど活発になるはず)"
    #tw = get_twitter()
    tweet_with_image(tw, "docs/images/hokkaido-fd.jpg", tw_body)
    #tw_body

In [ ]:
cov19data = []
# CSVデータを整形する
with codecs.open(data_file, encoding="shift-jis") as f:
    l = f.readline() # 先頭行をスキップ
    while l:
        l = f.readline().replace("\r\n", "")
        arr = l.split(',')
        if len(arr) == 24:
            tracked = blank2zero(arr[20])
            untracked = blank2zero(arr[21])
            unknown_rate = (untracked / max(1.0, (tracked + untracked))) * 100.0
            #print(tracked, untracked, unknown_rate)
            cov19data.append(
                [
                    dt(int(arr[1]), int(arr[2]), int(arr[3])), # date
                    blank2zero(arr[4]),  # tests
                    blank2zero(arr[6]),  # positive
                    blank2zero(arr[19]), # postive rate [%]
                    tracked, # tracked 濃厚接触
                    untracked,  # untracked 濃厚接触以外
                    unknown_rate # 経路不明率
                ]
            )

print("Total: {}".format(len(cov19data)))

In [ ]:
pos_rate_np = np.array(cov19data)
#print("{}".format(pos_rate_np))

In [ ]:
xbins = pos_rate_np[:, 0]
two_weeks_later = pd.date_range(xbins[-1] + td(days=1), xbins[-1] + td(days=days2pred)).to_pydatetime()
xbins_pred = xbins.tolist()
xbins_pred.extend(two_weeks_later)
X = np.arange(0, len(pos_rate_np[:, 0]))[:, np.newaxis]
X_pred = np.arange(0, len(xbins_pred))[:, np.newaxis]

y_test = get_gpr_predict(X, pos_rate_np[:, 1], X_pred, 10, 10, 10)
y_rate = get_gpr_predict(X, pos_rate_np[:, 3], X_pred, 10, 10, 10)
y_unkn = get_gpr_predict(X, pos_rate_np[:, 6], X_pred, 10, 10, 10)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=pos_rate_np[:, 1], mode='markers', name='検査人数',
                                       marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=moving_average(pos_rate_np[:, 1]), name='移動平均', opacity=0.5), 
                    secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_test, mode='lines', name='予測値',
                         line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=pos_rate_np[:, 3], name="陽性率[%]", opacity=0.5),
                         secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_rate, name="予測値",
                         line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins_pred[30], xbins_pred[-1]]),
    yaxis=dict(title='人数', range=[0, np.max(pos_rate_np[:, 1])]),
    yaxis2=dict(title='陽性率[%]', range=[0, np.max(pos_rate_np[:, 3])]),
    title='北海道 新型コロナ 検査人数/陽性率({})'.format(today_str),
)
show_and_save_plotly(fig, "hokkaido-rate.jpg", js=False, show=False)

In [ ]:
tw_body = "北海道 新型コロナ 検査人数/陽性率(" + today_str + " 時点)"
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
tweet_with_image(tw, "docs/images/hokkaido-rate.jpg", tw_body)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=pos_rate_np[:, 1], mode='markers', name='検査人数',
                                       marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=moving_average(pos_rate_np[:, 1]), name='移動平均', opacity=0.5), 
                    secondary_y=False)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_test, mode='lines', name='予測値',
                         line=dict(width=1)), secondary_y=False)
fig.add_trace(go.Bar(x=xbins, y=pos_rate_np[:, 6], name="経路不明率[%]", opacity=0.5),
                         secondary_y=True)
fig.add_trace(go.Scatter(x=xbins_pred, y=y_unkn, name="予測値",
                         line=dict(width=1)), secondary_y=True)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=1209600000.0, tickformat="%_m/%-d",
               range=[xbins_pred[30], xbins_pred[-1]]),
    yaxis=dict(title='人数', range=[0, np.max(pos_rate_np[:, 1])]),
    yaxis2=dict(title='経路不明率[%]', range=[0, np.max(pos_rate_np[:, 6])]),
    title='北海道 新型コロナ 検査人数/経路不明率({})'.format(today_str),
)
show_and_save_plotly(fig, "hokkaido-unknown.jpg", js=False, show=False)

In [ ]:
today_str = dt.now().isoformat()[:19].replace('T', ' ')
tw_body = "北海道 新型コロナ 検査人数/経路不明率(" + today_str + " 時点)"
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
tweet_with_image(tw, "docs/images/hokkaido-unknown.jpg", tw_body)

In [ ]:
tw_body = "北海道 新型コロナ 振興局別 罹患率[全期間] (" + today_str + ")"
imgname = "hokkaido-all.jpg"
# 振興局別感染者数
sub_poss = df.groupby('Code').size()
#print("Sub-pref Pos: {}".format(sub_poss))
vals = np.zeros(len(sub_prefs.keys()), dtype=float)
for i in range(len(vals)):
    if i not in sub_poss.index:
        sub_poss.loc[i] = 0
#print(sub_poss)
for i in sub_poss.index:
    #print(i, sub_prefs[i]['code'])
    vals[i] = (sub_poss[i] / sub_prefs[i]['total']) * 100.0
#print(vals)
make_hokkaido_choropleth(imgname, tw_body, vals, show=False)
#make_hokkaido_plotly(imgname, tw_body, vals)

In [ ]:
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido-hm.html "
tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
tw_body = "北海道 新型コロナ 振興局別 陽性者数[全期間] (" + today_str + ")"
imgname = "hokkaido-all-n.jpg"
vals = np.zeros(len(sub_prefs.keys()), dtype=float)
for i in range(len(vals)):
    if i not in sub_poss.index:
        sub_poss.loc[i] = 0
for i in sub_poss.index:
    vals[i] = sub_poss[i]
make_hokkaido_choropleth(imgname, tw_body, vals, show=False)
#make_hokkaido_plotly(imgname, tw_body, vals)

In [ ]:
#tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido.html "
#tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
dt_last1w = end_date - td(7)
dt_last2w = end_date - td(14)
print(end_date, dt_last1w, dt_last2w)
# 直近1週間
df_last1w = df[df['Date'] > dt_last1w]
sub_last1w = df_last1w.groupby('Code').size()
# 直近2週間
df_last2w = df[df['Date'] > dt_last2w]
sub_last2w = df_last2w.groupby('Code').size()
# 直近1週間陽性者数
sub_pos_last1w = np.zeros(len(sub_prefs))
cnt = 0
for i in sub_last1w.index.to_list():
    sub_pos_last1w[i] = sub_last1w.values[cnt]
    cnt +=1
print(sub_pos_last1w.astype(int))
# 直近2週間陽性者数
sub_pos_last2w = np.zeros(len(sub_prefs))
cnt = 0
for i in sub_last2w.index.to_list():
    sub_pos_last2w[i] = sub_last2w.values[cnt]
    cnt += 1
print(sub_pos_last2w.astype(int))

In [ ]:
# 計算式は「（直近7日間の新規陽性者数／その前7日間の新規陽性者数）^（平均世代時間／報告間隔）」
agt = 5 # 平均世代時間
ri = 7 #報告間隔
min_smpl = 5 # 最低サンプリング数
Rt = []
for i in np.arange(len(sub_pos_last1w)):
    div = (sub_pos_last2w[i] - sub_pos_last1w[i])
    if div == 0 and sub_pos_last1w[i] > 0:
        Rt.append(1.0) # 0から増加した場合は他地域からの流入
    elif div == 0:
        Rt.append(0.0) # 0を維持
    elif sub_pos_last2w[i] < min_smpl and sub_pos_last1w[i] < min_smpl:
        Rt.append(1) # サンプリングが少ない場合、1 と仮定
    else:
        r = (sub_pos_last1w[i] / div) ** (agt / ri)
        Rt.append(r)
print(Rt)

In [ ]:
tw_body = "北海道 新型コロナ 振興局別 実効再生産数[簡易計算] (" + today_str + ")"
imgname = "hokkaido-Rt.jpg"
make_hokkaido_choropleth(imgname, tw_body, Rt, show=False)
#make_hokkaido_plotly(imgname, tw_body, Rt)
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido-hm.html "
tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
tw_body = "北海道 新型コロナ 振興局別 直近2週間罹患率 (" + today_str + ")"
imgname = "hokkaido-2w.jpg"
vals = []
for k, v in sub_prefs.items():
    vals.append((sub_pos_last2w[k] / v['total']) * 100.0)
print(vals)
make_hokkaido_choropleth(imgname, tw_body, vals, show=False)
#make_hokkaido_plotly(imgname, tw_body, vals)
tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido-hm.html "
tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
if False:
    title = '北海道 新型コロナ 新規感染者数/累乗近似 (' + today_str + ')'
    xos = 240
    make_exp_fit_graph(tw,
        pos_rate_np[xos:,0], pos_rate_np[xos:, 2],
        title, "hokkaido-fit.jpg",
        "hokkaido-doubling-time.html", "hokkaido-fit.html")

In [ ]:
if False:
    title = '北海道 新型コロナ 新規移動平均/累乗近似 (' + today_str + ')'
    xos = 225
    xbins = np.array([i.to_pydatetime() for i in daily_new.index.tolist()])
    ybins = np.array(daily_new.rolling(ave_mov_days).mean().to_list())
    make_exp_fit_graph(tw,
        xbins[xos:], ybins[xos:],
        title, "hokkaido-fit-ave.jpg",
        "hokkaido-doubling-time-ave.html", "hokkaido-fit.html")

In [ ]:
# 新規感染者数 移動平均
mv_mean = daily_new.rolling(ave_mov_days).mean()
# 札幌平均気温 移動平均
x_mv = moving_average(np_weather[:, 1])
# 札幌絶対湿度 移動平均
y_mv = moving_average(np_weather[:, 5])
# 札幌 移動平均
z_mv = moving_average(np_weather[:, 2])
# 気象情報 日付
weather_dates = np_weather[:, 0]
#print(weather_dates[0], weather_dates[-1])

In [ ]:
# 詰め替え
sizes = []
xbins = []
ybins = []
zbins = []
colors = []
texts = []
dofs = 7 # = 報告日 - 感染日
mv_mean = mv_mean.dropna(how='all')
mv_max = np.max(mv_mean)
for k, v in mv_mean.items():
    ts = k.to_pydatetime()
    if ts < dt(2020, 5, 25, 0, 0, 0, 0): continue
    if ts in weather_dates:
        i = int(np.where(weather_dates == ts)[0]) - dofs
        ts_str = "{}".format(ts - td(days=dofs))
        msg = "%s =%d v:%.2f t:%.2f vh:%.2f f:%.2f" % \
        (ts_str[:10], i, v, x_mv[i], y_mv[i], z_mv[i])
        xbins.append(x_mv[i]); ybins.append(y_mv[i]); zbins.append(z_mv[i]);
        colors.append(v); sizes.append(max(5,v*15/mv_max))
        texts.append("%s: %.1f人" % (ts_str[:10], v)); 
        #print(msg)

In [ ]:
def gen_hokkaido_rel_graph(x, y, xtype, ytype, xlabel, ylabel, imgname):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        mode='markers', x=x, y=y, text=texts,
        marker=dict(
            opacity=0.85, size=sizes, color=colors,
            colorscale=[[0, 'rgb(59, 70, 222)'],
                        [.3, 'rgb(255, 255, 255)'],
                        [1, 'rgb(178, 10, 28)']],
            colorbar_title = '人数移動平均')))
    fig.update_layout(template='plotly_dark')
    tw_body = '北海道(札幌) {}/{}と感染日 ('.format(xtype, ytype) + \
        today_str[:13] + '時)'
    fig.update_layout(
        height=500, width=500, margin=dict(l=12, r=5, b=12, t=42),
        xaxis =dict(domain=[0, 1], showgrid=True, title=xlabel),
        yaxis =dict(domain=[0, 1], title=ylabel),
        title=tw_body,
        showlegend=False)
    show_and_save_plotly(fig, imgname, js=False)
    tw_body += " (2020/5/25以降) "
    tw_body += " https://geneasyura.github.io/cov19-hm/{} ".format("hokkaido-trh-tvh.html")
    tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
gen_hokkaido_rel_graph(
    xbins, ybins, "気温", "絶対湿度", 
    '札幌 平均気温 移動平均 [℃]',
    '札幌 容積絶対湿度 移動平均 [g/㎥]',
    "hokkaido-tvh.jpg")

In [ ]:
gen_hokkaido_rel_graph(
    xbins, zbins, "気温", "相対湿度", 
    '札幌 平均気温 移動平均 [℃]',
    '札幌 相対湿度 移動平均 [%RH]',
    "hokkaido-trh.jpg")

In [ ]:
def gen_hokkaido_rel_contour(x, y, xtype, ytype, xlabel, ylabel, imgname):
    fig = go.Figure()
    fig.add_trace(go.Contour(
        x=x, y=y, z=colors, text=texts,
        colorbar_title='感染人数',
        contours=dict(
            coloring='heatmap',
            showlabels=True,
            labelfont=dict(size=12,color='white')
        )))
    fig.update_layout(template='plotly_dark')
    tw_body = '北海道(札幌) 感染日の{}/{} ('.format(xtype, ytype) + \
        today_str[:13] + '時)'
    fig.update_layout(
        height=500, width=500, margin=dict(l=12, r=5, b=12, t=42),
        xaxis =dict(domain=[0, 1], showgrid=True, title=xlabel),
        yaxis =dict(domain=[0, 1], title=ylabel),
        title=tw_body,
        showlegend=False)
    show_and_save_plotly(fig, imgname, js=False)
    tw_body += " (2020/5/25以降) "
    tw_body += " https://geneasyura.github.io/cov19-hm/{} ".format("hokkaido-trh-tvh.html")
    tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
gen_hokkaido_rel_contour(
    xbins, ybins, "気温", "絶対湿度", 
    '札幌 平均気温 移動平均 [℃]',
    '札幌 容積絶対湿度 移動平均 [g/㎥]',
    "hokkaido-tvh-contour.jpg")

In [ ]:
gen_hokkaido_rel_contour(
    xbins, zbins, "気温", "相対湿度", 
    '札幌 平均気温 移動平均 [℃]',
    '札幌 相対湿度 移動平均 [%RH]',
    "hokkaido-trh-contour.jpg")